<a href="https://colab.research.google.com/github/gilhotraapoorva/DF/blob/main/finalised.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install -q kaggle
from google.colab import files
files.upload()

Saving kaggle (2).json to kaggle (2).json


{'kaggle (2).json': b'{"username":"apoorvaaaaaa","key":"d66784d4565981100258ecf8a750e6a2"}'}

In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets list


ref                                                        title                                               size  lastUpdated          downloadCount  voteCount  usabilityRating  
---------------------------------------------------------  -------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
sudarshan24byte/online-food-dataset                        Online Food Dataset                                  3KB  2024-03-02 18:50:30          14073        279  0.9411765        
bhavikjikadara/student-study-performance                   Student Study Performance                            9KB  2024-03-07 06:14:09           7323        109  1.0              
lovishbansal123/sales-of-a-supermarket                     Sales of a Supermarket                              36KB  2024-03-26 12:38:06           1277         27  1.0              
syedanwarafridi/vehicle-sales-data                         Vehicle Sales Data             

In [3]:
!kaggle competitions download -c deepfake-detection-challenge

100% 4.13G/4.13G [00:45<00:00, 160MB/s]
100% 4.13G/4.13G [00:45<00:00, 97.4MB/s]


In [ ]:
! mkdir train

In [23]:
! unzip deepfake-detection-challenge.zip

Archive:  deepfake-detection-challenge.zip
  inflating: sample_submission.csv   
  inflating: test_videos/aassnaulhq.mp4  
  inflating: test_videos/aayfryxljh.mp4  
  inflating: test_videos/acazlolrpz.mp4  
  inflating: test_videos/adohdulfwb.mp4  
  inflating: test_videos/ahjnxtiamx.mp4  
  inflating: test_videos/ajiyrjfyzp.mp4  
  inflating: test_videos/aktnlyqpah.mp4  
  inflating: test_videos/alrtntfxtd.mp4  
  inflating: test_videos/aomqqjipcp.mp4  
  inflating: test_videos/apedduehoy.mp4  
  inflating: test_videos/apvzjkvnwn.mp4  
  inflating: test_videos/aqrsylrzgi.mp4  
  inflating: test_videos/axfhbpkdlc.mp4  
  inflating: test_videos/ayipraspbn.mp4  
  inflating: test_videos/bcbqxhziqz.mp4  
  inflating: test_videos/bcvheslzrq.mp4  
  inflating: test_videos/bdshuoldwx.mp4  
  inflating: test_videos/bfdopzvxbi.mp4  
  inflating: test_videos/bfjsthfhbd.mp4  
  inflating: test_videos/bjyaxvggle.mp4  
  inflating: test_videos/bkcyglmfci.mp4  
  inflating: test_videos/bktkwbcawi.m

In [ ]:
import os

In [ ]:
TRAIN_DIR = '/content/train/train_sample_videos/'
TMP_DIR = '/content/train/tmp/'
ZIP_NAME = 'dfdc_train_faces_sample.zip'
METADATA_PATH = TRAIN_DIR + 'metadata.json'

SCALE = 0.25
N_FRAMES = None

In [ ]:
!pip install facenet-pytorch > /dev/null 2>&1
!apt install zip > /dev/null 2>&1

In [5]:
# import os
# import glob
# import json
# import torch
# import cv2
# from PIL import Image
# import numpy as np
# import pandas as pd
# from tqdm.notebook import tqdm
# from facenet_pytorch import MTCNN

# device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
# print(f'Running on device: {device}')

In [6]:
import multiprocessing as mp

INPUT_DIR = '/kaggle/input/'
SAVE_PATH = '/kaggle/working/resnet50.pth' # The location where the model should be saved.
PRETRAINED_MODEL_PATH = ''

N_FACES = 5
TEST_SIZE = 0.3
RANDOM_STATE = 123

BATCH_SIZE = 32
NUM_WORKERS = mp.cpu_count()

WARM_UP_EPOCHS = 10
WARM_UP_LR = 1e-4
FINE_TUNE_EPOCHS = 100
FINE_TUNE_LR = 1e-6

THRESHOLD = 0.5
EPSILON = 1e-7

here i'm using formulas according to bayesian neural network as few values i was geeting nan, to overcome that issue,I m using BNN

In [7]:
def calculate_f1(preds, labels):
    labels = np.array(labels, dtype=np.uint8)
    preds = (np.array(preds) >= THRESHOLD).astype(np.uint8)
    tp = np.count_nonzero(np.logical_and(labels, preds))
    tn = np.count_nonzero(np.logical_not(np.logical_or(labels, preds)))
    fp = np.count_nonzero(np.logical_not(labels)) - tn
    fn = np.count_nonzero(labels) - tp
    precision = tp / (tp + fp + EPSILON)
    recall = tp / (tp + fn + EPSILON)
    f1 = (2 * precision * recall) / (precision + recall + EPSILON)

    return f1

In [8]:
import tensorflow as tf

In [10]:
import tensorflow as tf
from tqdm import tqdm
import numpy as np

def train_the_model(model, criterion, optimizer, epochs, train_dataloader, val_dataloader, best_val_loss=1e7, best_val_logloss=1e7, save_the_best_on='val_loss'):

  losses = np.zeros(epochs)
  val_losses = np.zeros(epochs)
  loglosses = np.zeros(epochs)
  val_loglosses = np.zeros(epochs)
  f1_scores = np.zeros(epochs)
  val_f1_scores = np.zeros(epochs)
  best_model_state_dict = None
  best_optimizer_state_dict = None

  for epoch in range(epochs):
    batch_losses = []
    # Training loop within the epoch
    for sample_batched in tqdm(train_dataloader):
      with tf.GradientTape() as tape:
        # Make prediction
        y_pred = model(sample_batched['faces'])
        # Compute loss
        loss = criterion(y_pred, sample_batched['label'])
      batch_losses.append(loss.numpy())

      # Backpropagation and update weights
      grads = tape.gradient(loss, model.trainable_variables)
      optimizer.apply_gradients(zip(grads, model.trainable_variables))

      # Calculate and display training metrics
      # (F1 score calculation might need adjustment for TensorFlow)
      loglosses[epoch] = calculate_logloss(y_pred, sample_batched['label'])  # Assuming calculate_logloss is compatible with TensorFlow tensors
      f1_scores[epoch] = calculate_f1(y_pred.numpy(), sample_batched['label'].numpy())  # Assuming calculate_f1 works with NumPy arrays

    losses[epoch] = np.mean(batch_losses)

    # Validation loop
    val_batch_losses = []
    for sample_batched in val_dataloader:
      # Make prediction and compute loss
      y_pred = model(sample_batched['faces'])
      val_loss = criterion(y_pred, sample_batched['label'])
      val_batch_losses.append(val_loss.numpy())

      # Calculate and display validation metrics
      val_loglosses[epoch] = calculate_logloss(y_pred, sample_batched['label'])  # Assuming calculate_logloss is compatible with TensorFlow tensors
      val_f1_scores[epoch] = calculate_f1(y_pred.numpy(), sample_batched['label'].numpy())  # Assuming calculate_f1 works with NumPy arrays

    val_losses[epoch] = np.mean(val_batch_losses)
    print(f'loss: {losses[epoch]} | val loss: {val_losses[epoch]} | f1: {f1_scores[epoch]} | val f1: {val_f1_scores[epoch]} | log loss: {loglosses[epoch]} | val log loss: {val_loglosses[epoch]}')

    # Update best model/optimizer if validation loss improves
    if val_losses[epoch] < best_val_loss:
      best_val_loss = val_losses[epoch]
      if save_the_best_on == 'val_loss':
        print('Found a better checkpoint!')
        best_model_state_dict = model.get_weights()
        best_optimizer_state_dict = optimizer.get


    return losses, val_losses, loglosses, val_loglosses, f1_scores, val_f1_scores, best_val_loss, best_val_logloss, best_model_state_dict, best_optimizer_state_dict




In [11]:
def visualize_results(
    losses,
    val_losses,
    loglosses,
    val_loglosses,
    f1_scores,
    val_f1_scores
):
    fig = plt.figure(figsize=(16, 8))
    ax = fig.add_axes([0, 0, 1, 1])

    ax.plot(np.arange(1, len(losses) + 1), losses)
    ax.plot(np.arange(1, len(val_losses) + 1), val_losses)
    ax.set_xlabel('epoch', fontsize='xx-large')
    ax.set_ylabel('focal loss', fontsize='xx-large')
    ax.legend(
        ['loss', 'val loss'],
        loc='upper right',
        fontsize='xx-large',
        shadow=True
    )
    plt.show()


    fig = plt.figure(figsize=(16, 8))
    ax = fig.add_axes([0, 0, 1, 1])
    ax.plot(np.arange(1, len(loglosses) + 1), loglosses)
    ax.plot(np.arange(1, len(val_loglosses) + 1), val_loglosses)
    ax.set_xlabel('epoch', fontsize='xx-large')
    ax.set_ylabel('log loss', fontsize='xx-large')
    ax.legend(
        ['log loss', 'val log loss'],
        loc='upper right',
        fontsize='xx-large',
        shadow=True
    )
    plt.show()


    fig = plt.figure(figsize=(16, 8))
    ax = fig.add_axes([0, 0, 1, 1])

    ax.plot(np.arange(1, len(f1_scores) + 1), f1_scores)
    ax.plot(np.arange(1, len(val_f1_scores) + 1), val_f1_scores)
    ax.set_xlabel('epoch', fontsize='xx-large')
    ax.set_ylabel('f1 score', fontsize='xx-large')
    ax.legend(
        ['f1', 'val f1'],
        loc='upper left',
        fontsize='xx-large',
        shadow=True
    )
    plt.show()

In [13]:
import tensorflow as tf

class DeepfakeClassifier(tf.keras.Model):
  def __init__(self, encoder, in_channels=3, num_classes=1):
    super(DeepfakeClassifier, self).__init__()
    self.encoder = encoder

    # Modify input layer
    self.conv1 = tf.keras.layers.Conv2D(
        filters=64, kernel_size=7, strides=2, padding="same", input_shape=(None, None, in_channels), use_bias=False
    )

    # Modify output layer
    self.fc = tf.keras.layers.Dense(num_classes, activation="sigmoid")

  def call(self, x):
    x = self.conv1(x)
    x = self.encoder(x)
    return x

  def freeze_all_layers(self):
    for layer in self.layers:
      layer.trainable = False

  def freeze_middle_layers(self):
    self.freeze_all_layers()
    self.conv1.trainable = True
    self.fc.trainable = True

  def unfreeze_all_layers(self):
    for layer in self.layers:
      layer.trainable = True

In [15]:
class FaceDataset(tf.keras.utils.Sequence):
    def __init__(self, img_dirs, labels, n_faces=1, preprocess=None):
        self.img_dirs = img_dirs
        self.labels = labels
        self.n_faces = n_faces
        self.preprocess = preprocess

    def __len__(self):
        return len(self.img_dirs)

    def __getitem__(self, idx):
        img_dir = self.img_dirs[idx]
        label = self.labels[idx]

        face_paths = glob(f'{img_dir}/*.png')

        if len(face_paths) >= self.n_faces:
            sample = tf.random.choice(face_paths, self.n_faces, replace=False)
        else:
            sample = tf.random.choice(face_paths, self.n_faces, replace=True)

        faces = []
        for face_path in sample.numpy():
            face = cv2.imread(face_path, cv2.IMREAD_COLOR)  # Ensure color reading
            face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)

            # TensorFlow-specific image loading and preprocessing
            face = load_img(face, target_size=(face.shape[0], face.shape[1]))
            face = img_to_array(face)
            if self.preprocess is not None:
                face = self.preprocess(tf.expand_dims(face, axis=0))['image'][0]

            faces.append(face)

        faces = np.concatenate(faces, axis=-1)  # Concatenate along channels
        faces = tf.convert_to_tensor(faces.transpose(2, 0, 1))  # Efficient transposing

        return {'faces': faces, 'label': tf.cast(label, tf.float32)}



class FaceValDataset(tf.data.Dataset):

    def __init__(self, img_dirs, labels, n_faces=1, preprocess=None):
        self.img_dirs = img_dirs
        self.labels = labels
        self.n_faces = n_faces
        self.preprocess = preprocess

    def _parse_function(self, sample):
        img_dir, label = sample

        face_paths = tf.strings.glob(f'{img_dir}/*.png')

        # Extract face indices efficiently using tf.strings
        face_indices = tf.strings.split(
            tf.strings.split(face_paths, sep='/')[-1], sep='.'
        )[0][..., 0]
        max_idx = tf.cast(tf.math.reduce_max(tf.strings.to_number(face_indices, tf.uint32)), tf.uint32)

        # Efficiently select faces with a stride
        strides = tf.range(self.n_faces) * tf.cast(
            tf.math.ceil(max_idx + 1 / (self.n_faces**2)), tf.float32
        )
        sample_indices = tf.cast(tf.math.floor(strides), tf.int32)

        selected_paths = []
        for idx in sample_indices:
            paths = tf.strings.glob(f'{img_dir}/{idx}*.png')
            selected_paths.append(paths)

        selected_paths = tf.concat(selected_paths, axis=0)
        selected_paths = tf.gather(selected_paths, tf.range(self.n_faces))  # Top n_faces

        faces = []
        for selected_path in selected_paths:
            img = tf.io.read_file(selected_path)
            img = tf.image.decode_png(img, channels=3)  # Ensure color reading
            img = tf.cast(img, tf.float32)
            img = tf.image.resize(img, (img.shape[0], img.shape[1]))  # Resize if needed
            img = tf.cast(tf.image.rgb_to_hsv(img), tf.float32)  # Example preprocessing

            faces.append(img)

        if self.preprocess is not None:
            faces = tf.stack(faces)
            faces = self.preprocess(tf.expand_dims(faces, axis=0))['image']
            faces = tf.squeeze(faces, axis=0)  # Remove unnecessary dimension

        faces = tf.transpose(tf.concat(faces, axis=-1), perm=[2, 0, 1])

        return {'faces': faces, 'label': tf.cast(label, tf.float32)}

    def __iter__(self):
        dataset = tf.data.Dataset.from_tensor_slices((self.img_dirs, self.labels))
        return dataset.map(self._parse_function)



In [17]:
import tensorflow as tf
from tensorflow.keras import backend as K

class FocalLoss(tf.keras.losses.Loss):

    def __init__(self, gamma=2.0, alpha=None, from_logits=True):
        super(FocalLoss, self).__init__(reduction=tf.keras.losses.Reduction.SUM_OVER_BATCH_MEAN)
        self.gamma = gamma
        self.alpha = alpha
        self.from_logits = from_logits

    def call(self, y_true, y_pred):
        """Focal loss with support for weighting and handling logits."""
        if self.from_logits:
            y_pred = tf.nn.sigmoid(y_pred)  # Convert logits to probabilities

        pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
        pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))

        if self.alpha is not None:
            alpha_t = tf.where(tf.equal(y_true, 1), self.alpha, 1.0 - self.alpha)
            alpha_f = tf.where(tf.equal(y_true, 0), self.alpha, 1.0 - self.alpha)
            cl = (-alpha_t * K.log(pt_1)) * ((1 - pt_1) ** self.gamma)
            cf = (-alpha_f * K.log(1.0 - pt_0)) * (pt_0 ** self.gamma)
        else:
            cl = (-K.log(pt_1)) * ((1 - pt_1) ** self.gamma)
            cf = (-K.log(1.0 - pt_0)) * (pt_0 ** self.gamma)

        loss = cl + cf
        return self.reduction(loss)


In [26]:
import tensorflow.io.gfile as gfile
all_train_dirs = gfile.glob(INPUT_DIR + '/train_sample_videos')
all_train_dirs = sorted(all_train_dirs)
for i, train_dir in enumerate(all_train_dirs):
  tf.print('[{:02}]'.format(i), train_dir)


In [43]:
import tensorflow.io.gfile as gfile
import tensorflow as tf
import pandas as pd

def _parse_csv(filename):
  """Reads a CSV file using tf.data.TextLineDataset and pandas."""
  try:
    dataset = tf.data.TextLineDataset(filename)
    df = pd.read_csv(dataset.skip(1))  # Skip header row
    return df
  except Exception as e:
    print(f"Error reading CSV: {filename} - {e}")
    return None  # Or handle the error differently

# Ensure path consistency (replace with your actual directory)
INPUT_DIR = '/content'

# Find all train directories within the specified path
all_train_dirs = sorted(gfile.glob(f'{INPUT_DIR}/train_sample_videos/*'))

# Print the contents of all_train_dirs to check for directories
print("Found directories:")
for train_dir in all_train_dirs:
  print(train_dir)

# Check if any directories were found
if not all_train_dirs:
  print("No directories found in train_sample_videos. Check the path or file structure.")
  exit()  # Terminate execution if no directories found

import pandas as pd
all_dataframes = []
for train_dir in all_train_dirs:
  #print(train_dir)
  # metadata_path = os.path.join(train_dir, 'metadata.csv')
  metadata_path = '/content/metadata.csv'
  # df = _parse_csv(metadata_path)
  df = pd.read_csv(metadata_path,names=['filename','label','split','original'],header=1)
  # print(df.head())
  # print(df.columns)
  if df is not None:
    df['path'] = df['filename'].apply(lambda x: os.path.join(train_dir, x.split('.')[0]))
    all_dataframes.append(df)
  else:
    print(f"Error reading CSV: {metadata_path}")

if all_dataframes:
  train_df = pd.concat(all_dataframes, ignore_index=True, sort=False)
  print("Successfully loaded dataframes. Now you can use the 'train_df' for further processing.")
else:
  print("No DataFrames to concatenate. Check for errors or empty directories.")


Found directories:
/content/train_sample_videos/aagfhgtpmv.mp4
/content/train_sample_videos/aapnvogymq.mp4
/content/train_sample_videos/abarnvbtwb.mp4
/content/train_sample_videos/abofeumbvv.mp4
/content/train_sample_videos/abqwwspghj.mp4
/content/train_sample_videos/acifjvzvpm.mp4
/content/train_sample_videos/acqfdwsrhi.mp4
/content/train_sample_videos/acxnxvbsxk.mp4
/content/train_sample_videos/acxwigylke.mp4
/content/train_sample_videos/aczrgyricp.mp4
/content/train_sample_videos/adhsbajydo.mp4
/content/train_sample_videos/adohikbdaz.mp4
/content/train_sample_videos/adylbeequz.mp4
/content/train_sample_videos/aelfnikyqj.mp4
/content/train_sample_videos/aelzhcnwgf.mp4
/content/train_sample_videos/aettqgevhz.mp4
/content/train_sample_videos/aevrfsexku.mp4
/content/train_sample_videos/afoovlsmtx.mp4
/content/train_sample_videos/agdkmztvby.mp4
/content/train_sample_videos/agqphdxmwt.mp4
/content/train_sample_videos/agrmhtjdlk.mp4
/content/train_sample_videos/ahbweevwpv.mp4
/content/trai

In [44]:
train_df

,filename,label,split,original,path
0,aapnvogymq.mp4,FAKE,train,jdubbvfswz.mp4,/content/train_sample_videos/aagfhgtpmv.mp4/aa...
1,abarnvbtwb.mp4,REAL,train,NaN,/content/train_sample_videos/aagfhgtpmv.mp4/ab...
2,abofeumbvv.mp4,FAKE,train,atvmxvwyns.mp4,/content/train_sample_videos/aagfhgtpmv.mp4/ab...
3,abqwwspghj.mp4,FAKE,train,qzimuostzz.mp4,/content/train_sample_videos/aagfhgtpmv.mp4/ab...
4,acifjvzvpm.mp4,FAKE,train,kbvibjhfzo.mp4,/content/train_sample_videos/aagfhgtpmv.mp4/ac...
...,...,...,...,...,...
159994,etejaapnxh.mp4,FAKE,train,wtreibcmgm.mp4,/content/train_sample_videos/metadata.json/ete...
159995,etmcruaihe.mp4,FAKE,train,afoovlsmtx.mp4,/content/train_sample_videos/metadata.json/etm...
159996,etohcvnzbj.mp4,FAKE,train,bdnaqemxmr.mp4,/content/train_sample_videos/metadata.json/eto...
159997,eudeqjhdfd.mp4,REAL,train,NaN,/content/train_sample_videos/metadata.json/eud...


In [45]:
train_df.head()


,filename,label,split,original,path
0,aapnvogymq.mp4,FAKE,train,jdubbvfswz.mp4,/content/train_sample_videos/aagfhgtpmv.mp4/aa...
1,abarnvbtwb.mp4,REAL,train,NaN,/content/train_sample_videos/aagfhgtpmv.mp4/ab...
2,abofeumbvv.mp4,FAKE,train,atvmxvwyns.mp4,/content/train_sample_videos/aagfhgtpmv.mp4/ab...
3,abqwwspghj.mp4,FAKE,train,qzimuostzz.mp4,/content/train_sample_videos/aagfhgtpmv.mp4/ab...
4,acifjvzvpm.mp4,FAKE,train,kbvibjhfzo.mp4,/content/train_sample_videos/aagfhgtpmv.mp4/ac...


remove corrupt videos

In [46]:
# Remove empty folders
train_df = train_df[train_df['path'].map(lambda x: os.path.exists(x))]

In [48]:
train_df.head()


,filename,label,split,original,path


In [72]:
valid_train_df = pd.DataFrame(columns=['filename', 'label', 'split', 'original', 'path'])

# for row_idx, row in tqdm(train_df.iterrows()):
for row_idx in tqdm(train_df.index):
    row = train_df.loc[row_idx]
    img_dir = row['path']
    face_paths = glob.glob(f'{img_dir}/*.png')

    if len(face_paths) >= N_FACES: # Satisfy the minimum requirement for the number of faces
        face_indices = [
            path.split('/')[-1].split('.')[0].split('_')[0]
            for path in face_paths
        ]
        max_idx = np.max(np.array(face_indices, dtype=np.uint32))

        selected_paths = []

        for i in range(N_FACES):
            stride = int((max_idx + 1)/(N_FACES**2))
            sample = np.linspace(i*stride, max_idx + i*stride, N_FACES).astype(int)

            # Get faces
            for idx in sample:
                paths = glob.glob(f'{img_dir}/{idx}*.png')

                selected_paths.extend(paths)
                if len(selected_paths) >= N_FACES: # Get enough faces
                    break

            if len(selected_paths) >= N_FACES: # Get enough faces
                valid_train_df = valid_train_df.append(row, ignore_index=True)
                break


0it [00:00, ?it/s]


In [73]:
valid_train_df.head()

,filename,label,split,original,path


In [51]:
valid_train_df['label'].replace({'FAKE': 1, 'REAL': 0}, inplace=True)

In [52]:
valid_train_df.head()

,filename,label,split,original,path


In [53]:
label_count = valid_train_df.groupby('label').count()['filename']
print(label_count)

Series([], Name: filename, dtype: int64)


In [54]:
X = valid_train_df['path'].to_numpy()
y = valid_train_df['label'].to_numpy()

In [70]:
try:
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE, stratify=y)
except ValueError as e:
    print(f"Error: Empty dataset. Skipping data splitting. {e}")
    # Handle the empty data situation here (e.g., collect more data)


Error: Empty dataset. Skipping data splitting. With n_samples=0, test_size=0.3 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.


In [68]:
from tensorflow.keras.layers import Normalization

# Create a Normalization layer with specified mean and std
normalization_layer = Normalization(mean=[0.485, 0.456, 0.406], variance=[0.229**2, 0.224**2, 0.225**2])

# Apply normalization during model definition
model = tf.keras.Sequential([
  normalization_layer,
  # Other layers in your model
])


In [ ]:
train_dataset = FaceDataset(
    img_dirs=X_train,
    labels=y_train,
    n_faces=N_FACES,
    preprocess=preprocess
)
val_dataset = FaceValDataset(
    img_dirs=X_val,
    labels=y_val,
    n_faces=N_FACES,
    preprocess=preprocess
)

train_dataloader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=NUM_WORKERS
)
val_dataloader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=NUM_WORKERS
)

In [ ]:
if os.path.exists(PRETRAINED_MODEL_PATH):
    encoder = resnet18(pretrained=False)
    classifier = DeepfakeClassifier(encoder=encoder, in_channels=3*N_FACES, num_classes=1)
    state = torch.load(PRETRAINED_MODEL_PATH, map_location=lambda storage, loc: storage)
    classifier.load_state_dict(state['state_dict'])
else:
    encoder = resnet18(pretrained=True)
    classifier = DeepfakeClassifier(encoder=encoder, in_channels=3*N_FACES, num_classes=1)

classifier.to(device)
classifier.train()

In [ ]:
criterion = FocalLoss()

In [ ]:
losses = np.zeros(WARM_UP_EPOCHS + FINE_TUNE_EPOCHS)
val_losses = np.zeros(WARM_UP_EPOCHS + FINE_TUNE_EPOCHS)
loglosses = np.zeros(WARM_UP_EPOCHS + FINE_TUNE_EPOCHS)
val_loglosses = np.zeros(WARM_UP_EPOCHS + FINE_TUNE_EPOCHS)
f1_scores = np.zeros(WARM_UP_EPOCHS + FINE_TUNE_EPOCHS)
val_f1_scores = np.zeros(WARM_UP_EPOCHS + FINE_TUNE_EPOCHS)

if os.path.exists(PRETRAINED_MODEL_PATH):
    best_val_loss = state['best_val_loss']
else:
    best_val_loss = 1e7

if os.path.exists(PRETRAINED_MODEL_PATH):
    best_val_logloss = state['best_val_logloss']
else:
    best_val_logloss = 1e7

In [ ]:
classifier.freeze_middle_layers()
warmup_optimizer = optim.Adam(filter(lambda p: p.requires_grad, classifier.parameters()), lr=WARM_UP_LR)
if os.path.exists(PRETRAINED_MODEL_PATH) and 'warmup_optimizer' in state.keys():
    warmup_optimizer.load_state_dict(state['warmup_optimizer'])

In [ ]:
losses[:WARM_UP_EPOCHS], val_losses[:WARM_UP_EPOCHS], \
loglosses[:WARM_UP_EPOCHS], val_loglosses[:WARM_UP_EPOCHS], \
f1_scores[:WARM_UP_EPOCHS], val_f1_scores[:WARM_UP_EPOCHS], \
best_val_loss, best_val_logloss, \
best_model_state_dict, best_optimizer_state_dict \
= train_the_model(
    model=classifier,
    criterion=criterion,
    optimizer=warmup_optimizer,
    epochs=WARM_UP_EPOCHS,
    train_dataloader=train_dataloader,
    val_dataloader=val_dataloader,
    best_val_loss=best_val_loss,
    best_val_logloss=best_val_logloss,
    save_the_best_on='val_logloss'
)

# Save the best checkpoint.
if best_model_state_dict is not None:
    state = {
        'state_dict': best_model_state_dict,
        'warmup_optimizer': best_optimizer_state_dict,
        'best_val_loss': best_val_loss,
        'best_val_logloss': best_val_logloss
    }
    torch.save(state, SAVE_PATH)

In [ ]:
visualize_results(
    losses=losses[:WARM_UP_EPOCHS],
    val_losses=val_losses[:WARM_UP_EPOCHS],
    loglosses=loglosses[:WARM_UP_EPOCHS],
    val_loglosses=val_loglosses[:WARM_UP_EPOCHS],
    f1_scores=f1_scores[:WARM_UP_EPOCHS],
    val_f1_scores=val_f1_scores[:WARM_UP_EPOCHS]
)

In [ ]:
classifier.unfreeze_all_layers()


In [ ]:
finetune_optimizer = optim.Adam(filter(lambda p: p.requires_grad, classifier.parameters()), lr=FINE_TUNE_LR)
if os.path.exists(PRETRAINED_MODEL_PATH) and 'finetune_optimizer' in state.keys() and WARM_UP_EPOCHS == 0:
    finetune_optimizer.load_state_dict(state['finetune_optimizer'])

In [ ]:
losses[WARM_UP_EPOCHS:WARM_UP_EPOCHS+FINE_TUNE_EPOCHS], val_losses[WARM_UP_EPOCHS:WARM_UP_EPOCHS+FINE_TUNE_EPOCHS], \
loglosses[WARM_UP_EPOCHS:WARM_UP_EPOCHS+FINE_TUNE_EPOCHS], val_loglosses[WARM_UP_EPOCHS:WARM_UP_EPOCHS+FINE_TUNE_EPOCHS], \
f1_scores[WARM_UP_EPOCHS:WARM_UP_EPOCHS+FINE_TUNE_EPOCHS], val_f1_scores[WARM_UP_EPOCHS:WARM_UP_EPOCHS+FINE_TUNE_EPOCHS], \
best_val_loss, best_val_logloss, \
best_model_state_dict, best_optimizer_state_dict \
= train_the_model(
    model=classifier,
    criterion=criterion,
    optimizer=finetune_optimizer,
    epochs=FINE_TUNE_EPOCHS,
    train_dataloader=train_dataloader,
    val_dataloader=val_dataloader,
    best_val_loss=best_val_loss,
    best_val_logloss=best_val_logloss,
    save_the_best_on='val_logloss'
)

if best_model_state_dict is not None:
    state = {
        'state_dict': best_model_state_dict,
        'finetune_optimizer': best_optimizer_state_dict,
        'best_val_loss': best_val_loss,
        'best_val_logloss': best_val_logloss
    }

    torch.save(state, SAVE_PATH)

In [ ]:
visualize_results(
    losses=losses,
    val_losses=val_losses,
    loglosses=loglosses,
    val_loglosses=val_loglosses,
    f1_scores=f1_scores,
    val_f1_scores=val_f1_scores
)